<left><h1> Extra project (4 points)
</h1><left>
    
<left><h2> Due December 5 11:59pm (Hard deadline)
</h2><left>    

In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets

First, we load the data set. 

In [2]:
iris=datasets.load_iris()

In [3]:
A=iris.data
# Sepal length, Sepal width, Petal length, Petal width

In [4]:
B=iris.target
# setora, vericolor, virginica

In [5]:
X=iris.data[:,[2,3]]
y=iris.target

In [ ]:
# 1. Shuffling

In [ ]:
# Generate integer from 0 to 149
arr1=np.arange(0, 150)
# Shuffle those integers randomly
np.random.shuffle(arr1)
arr1
# Note that arr1 may chage every time you run this cell due to randomness

In [7]:
# Then, we can obtain the shuffled data set by using the shuffled arrary.
Xnew=X[arr1,:]
ynew=y[arr1]

<h2>Task 1

In [ ]:
# 2. Preprocess the data

In [ ]:
# 2-1. Scaling

In [10]:
def scaling(X_org):
    # X_org: shuffled but non-scaled data set: X_org n by m matrix (n: example, m: feature)
    # Xmean is the mean value per each column:  Xmean 1 by m matrix
    # Xstd is the standard deviation per each column:  Xstd 1 by m matrix
    # X_sc: the scaled data set
    
    # Code starts  
    Xmean = X_org.mean(axis=0)
    Xstd = X_org.std(axis=0)
    # Code ends 
    
    X_sc = (X_org - Xmean) / Xstd
    
    return X_sc    

In [ ]:
X_sc=scaling(Xnew)
X_sc

In [ ]:
# 2-2 Split the data sent into the training set and test set

In [ ]:
# Transpose the data set, X_sc and ynew . 
# Code starts 
Xd = X_sc.T
yd = ynew.T
# Code ends

In [ ]:
# Modify the dimension of yd for matrix multiplication (np.matmul)
yd=np.reshape(ynew,(1,ynew.shape[0]))

In [ ]:
yd.shape

In [ ]:
# Here we are interested in finding 'Label 2'. We use the one-vs-all approach to allow binary classification.
# yd takes 1 when the previous yd is 2, otherwise yd takes 0.

# Code starts (one or two lines) 
yd = (yd == 2).astype(int)
# Code ends 

In [ ]:
yd

In [ ]:
# Then we split the data set for training (0-119) and testing (120-149)
X_train=Xd[:,:120]
X_test=Xd[:,120:]
y_train=yd[:,:120]
y_test=yd[:,120:]

In [ ]:
# 3. Logistic regression

In [ ]:
# 3-1 sigmoid function

In [12]:
import numpy as np

def sigmoid(z):
    # z is the input value (scalar, vector, matrix)
    # s is the output value (scalar, vector, matrix)
    # Code starts 
    s = 1 / (1 + np.exp(-z))
    # Code ends 
    return s


In [ ]:
# 4-Initializing parameters

In [13]:
# Initialize parameters with zero values
def initialize(ndim):
    # ndim is the number of features: m 
    # bias is a scalar: theta_0 
    # wgt is a ndim by 1 matrix ([ndim,1], dtype=float): (theta_1,..., theta_m) 
    # Code starts 
    wgt = np.zeros((ndim, 1), dtype=float)
    bias = 0.0
    # Code ends 
    return wgt, bias

In [ ]:
# 5-Foward propagation

In [14]:
def forward_propagate(wgt, bias, X):
    # X is the transpose of X_sc, m by n matrix
    # You can use np.matmul() for matrix multiplication to calculate z
    # aout is a 1 by n matrix
    
    # Code starts 
    z = np.matmul(wgt.T, X) + bias
    # Code ends 
    
    aout = sigmoid(z)
    return aout


In [ ]:
# 6-Calculating cost function and gradient

In [15]:
def cost_grad(wgt, bias, X_train, y_train, C):
    # X_train, y_train are the transpose of scaled training matrices. (m by n matrices)
    # C is the inverse of lambda: regularized parameter.
    # cost is the cost function values (J); scalar
    # dw is dJ/dtheta, m by 1 matrix
    # db is dJ/db
    # grads is the dictionary that contains dw and db
    m = X_train.shape[0]
    aout = forward_propagate(wgt, bias, X_train)
    
    # Code starts 
    cost = -(1/m) * np.sum(y_train * np.log(aout) + (1 - y_train) * np.log(1 - aout)) + (C/(2 * m)) * np.sum(wgt**2)
    
    dz = aout - y_train
    dw = (1/m) * np.matmul(X_train, dz.T) + (C/m) * wgt
    db = (1/m) * np.sum(dz)
    # Code ends 
    
    cost = np.squeeze(np.array(cost))
    grads = {"dw": dw, "db": db}
    
    return grads, cost


In [ ]:
# 7-Back propagation

In [17]:
def back_propagate(wgt, bias, X_train, y_train, C=100, learn_rate=0.01, print_cost=True):
    # learn_rate is the learning rate:
    grads, cost = cost_grad(wgt, bias, X_train, y_train, C)
    # Retrieve derivatives from grads
    dw = grads["dw"]
    db = grads["db"]
    
    # Update the weights w (updated) = w (old) - alpha * dw
    # Code starts 
    wgt = wgt - learn_rate * dw
    bias = bias - learn_rate * db
    # Code ends 
    
    # Print the cost every 100 training iterations
    if print_cost:
        print("Cost after iteration %f" % cost)
                
    # params is the dictionary that stores the weights
    params = {"w": wgt, "b": bias}
    grads = {"dw": dw, "db": db}    
    return params, grads, cost   

In [ ]:
def predict(wgt, bias, X_test):
    # X_test is a m by n matrix for testing
    # y_pred is a 1 by n matrix
    aout = forward_propagate(wgt, bias, X_test)
    
    # Code starts 
    y_pred = (aout > 0.5).astype(int)
    # Code ends 
    
    return y_pred

In [ ]:
# 9-Get them algother for the training

In [19]:
num_iterations=500
wt, bias = initialize(X_train.shape[0])
for i in range(num_iterations):
    params, grads, costs = back_propagate(wt, bias, X_train, y_train, C=100, learn_rate=0.01)
    wt=params['w']
    bias=params['b']
    print(wt)

In [ ]:
# 9-1 Make the figures of evolutiong of the cost function (iteration vs cost) in the cases of
# C=100, C=0.0001, learn_rate=0.01, learn_rate=1.0, learn_rate=0.0001

In [ ]:
import matplotlib.pyplot as plt

# Assume you have initialized wgt and bias appropriately
ndim = X_train.shape[1]
wgt, bias = initialize(ndim)

# Define hyperparameters
C_values = [100, 0.0001]
learn_rate_values = [0.01, 1.0, 0.0001]

# Number of iterations
num_iterations = 1000

# Initialize a dictionary to store costs for each combination of parameters
costs_dict = {}

# Iterate over combinations of C and learning rates
for C in C_values:
    for learn_rate in learn_rate_values:
        # Initialize parameters
        wgt, bias = initialize(ndim)
        
        # Lists to store costs for plotting
        costs = []
        
        # Training loop
        for i in range(num_iterations):
            # Perform backpropagation and update parameters
            params, grads, cost = back_propagate(wgt, bias, X_train, y_train, C=C, learn_rate=learn_rate, print_cost=False)
            wgt = params["w"]
            bias = params["b"]
            
            # Append cost to list
            costs.append(cost)
        
        # Store costs in the dictionary
        costs_dict[(C, learn_rate)] = costs

# Plotting
plt.figure(figsize=(12, 8))

for (C, learn_rate), costs in costs_dict.items():
    label = f'C={C}, LR={learn_rate}'
    plt.plot(range(num_iterations), costs, label=label)

plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.title('Evolution of Cost Function for Different Hyperparameters')
plt.legend()
plt.show()

In [ ]:
# 10-Predict for the testing

In [ ]:
y_pred=predict(wt, bias, X_test)

In [ ]:
# 10-2. Show your accuracy for six cases

In [ ]:
# Initialize parameters
ndim = X_train.shape[1]
wgt, bias = initialize(ndim)

# Define hyperparameters
C_values = [100, 0.0001]
learn_rate_values = [0.01, 1.0, 0.0001]

# Number of iterations
num_iterations = 1000

# Dictionary to store accuracies for each combination of parameters
accuracies_dict = {}

# Iterate over combinations of C and learning rates
for C in C_values:
    for learn_rate in learn_rate_values:
        # Initialize parameters
        wgt, bias = initialize(ndim)
        
        # Training loop
        for i in range(num_iterations):
            # Perform backpropagation and update parameters
            params, grads, cost = back_propagate(wgt, bias, X_train, y_train, C=C, learn_rate=learn_rate, print_cost=False)
            wgt = params["w"]
            bias = params["b"]
        
        # Make predictions on the test set
        y_pred = predict(wgt, bias, X_test)
        
        # Calculate accuracy
        accuracy = np.mean(y_pred == y_test)
        
        # Store accuracy in the dictionary
        accuracies_dict[(C, learn_rate)] = accuracy

# Display accuracies
for (C, learn_rate), accuracy in accuracies_dict.items():
    print(f'C={C}, LR={learn_rate}: Accuracy = {accuracy}')

<h2>Task 2

In [ ]:
# 11-Compare your results with sklearn (e.g., C=100, C=0.0001)

In [50]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

In [51]:
# Create an object-Write the code
lr=LogisticRegression(C=100,random_state=1)

In [ ]:
# 11-1. Run the cases of C=100 and C=0.001 for training followed by testing

In [ ]:
# Initialize parameters
ndim = X_train.shape[1]
wgt, bias = initialize(ndim)

# Define hyperparameters
C_values = [100, 0.001]
learn_rate = 0.01
num_iterations = 1000

# Dictionary to store results
results_dict = {}

# Iterate over different values of C
for C in C_values:
    # Training
    for i in range(num_iterations):
        # Perform backpropagation and update parameters
        params, grads, cost = back_propagate(wgt, bias, X_train, y_train, C=C, learn_rate=learn_rate, print_cost=False)
        wgt = params["w"]
        bias = params["b"]

    # Testing
    y_pred = predict(wgt, bias, X_test)
    
    # Calculate accuracy
    accuracy = np.mean(y_pred == y_test)
    
    # Store results in the dictionary
    results_dict[C] = {"wgt": wgt, "bias": bias, "accuracy": accuracy}

# Display results
for C, result in results_dict.items():
    print(f'C={C}: Accuracy = {result["accuracy"]}')


In [ ]:
# 11-2 Compare the results of prediction (test set) with your code in terms of accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Assuming X_train, y_train, X_test, and y_test are defined

# Train logistic regression model with C=100
model_c100 = LogisticRegression(C=100)
model_c100.fit(X_train, y_train)
y_pred_c100 = model_c100.predict(X_test)
accuracy_c100 = accuracy_score(y_test, y_pred_c100)
print(f'Accuracy for C=100 (scikit-learn): {accuracy_c100}')

# Train logistic regression model with C=0.001
model_c0_001 = LogisticRegression(C=0.001)
model_c0_001.fit(X_train, y_train)
y_pred_c0_001 = model_c0_001.predict(X_test)
accuracy_c0_001 = accuracy_score(y_test, y_pred_c0_001)
print(f'Accuracy for C=0.001 (scikit-learn): {accuracy_c0_001}')

# Compare with your logistic regression results
your_accuracy_c100 = results_dict[100]["accuracy"]
your_accuracy_c0_001 = results_dict[0.001]["accuracy"]

print(f'Your accuracy for C=100: {your_accuracy_c100}')
print(f'Your accuracy for C=0.001: {your_accuracy_c0_001}')

In [ ]:
# 11-3 Visualize of decision boundaries from the sklearn results and your code as you did in the previous labs.

In [ ]:
# Function to plot decision boundaries
def plot_decision_boundary(model, X, y, title):
    h = .02  # Step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1

    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Paired)
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

# Assuming X_train, y_train, X_test, y_test are defined

# Train logistic regression model with C=100 using scikit-learn
model_c100_sklearn = LogisticRegression(C=100)
model_c100_sklearn.fit(X_train, y_train)
plot_decision_boundary(model_c100_sklearn, X_train, y_train, 'Decision Boundary - C=100 (scikit-learn)')

# Train logistic regression model with C=100 using your code
model_c100_your_code = LogisticRegression(C=100)
model_c100_your_code.coef_ = results_dict[100]["wgt"].T
model_c100_your_code.intercept_ = results_dict[100]["bias"]
plot_decision_boundary(model_c100_your_code, X_train, y_train, 'Decision Boundary - C=100 (your code)')

# Train logistic regression model with C=0.001 using scikit-learn
model_c0_001_sklearn = LogisticRegression(C=0.001)
model_c0_001_sklearn.fit(X_train, y_train)
plot_decision_boundary(model_c0_001_sklearn, X_train, y_train, 'Decision Boundary - C=0.001 (scikit-learn)')

# Train logistic regression model with C=0.001 using your code
model_c0_001_your_code = LogisticRegression(C=0.001)
model_c0_001_your_code.coef_ = results_dict[0.001]["wgt"].T
model_c0_001_your_code.intercept_ = results_dict[0.001]["bias"]
plot_decision_boundary(model_c0_001_your_code, X_train, y_train, 'Decision Boundary - C=0.001 (your code)')
